In [67]:
import pandas as pd
import numpy as np

In [68]:
#对训练数据和测试数据已进行处理，选择了pcall、Sex、Age、Fare四个参数变量
#导入数据集
#导入训练数据集
train = pd.read_csv('./train.csv')
#导入测试数据集
test = pd.read_csv('./test.csv')
#导入测试数据集标签
test_y = pd.read_csv('./submission.csv')


In [69]:
train

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0
...,...,...,...,...,...
886,2,male,27.0,13.0000,0
887,1,female,19.0,30.0000,1
888,3,female,NaN,23.4500,0
889,1,male,26.0,30.0000,1


In [70]:
test

,Pclass,Sex,Age,Fare
0,3,male,34.5,7.8292
1,3,female,47.0,7.0000
2,2,male,62.0,9.6875
3,3,male,27.0,8.6625
4,3,female,22.0,12.2875
...,...,...,...,...
413,3,male,NaN,8.0500
414,1,female,39.0,108.9000
415,3,male,38.5,7.2500
416,3,male,NaN,8.0500


In [71]:
#合并数据集用来对两个数据集进行数据清洗
full = train.append(test,ignore_index=True)
full

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0.0
1,1,female,38.0,71.2833,1.0
2,3,female,26.0,7.9250,1.0
3,1,female,35.0,53.1000,1.0
4,3,male,35.0,8.0500,0.0
...,...,...,...,...,...
1304,3,male,NaN,8.0500,NaN
1305,1,female,39.0,108.9000,NaN
1306,3,male,38.5,7.2500,NaN
1307,3,male,NaN,8.0500,NaN


In [ ]:
#数据预处理
#Age、Fare均存在缺失数据，缺失数据需要进行数据清洗处理；
#缺失值处理常用方法：如果是数值类型，用平均值取代；如果是分类数据，用最常见的类别取代；使用模型预测缺失值，例如：K-NN。
#对于Age（年龄）和Fare（船票价格）两个数值类型字段，缺失的部分采用最简单的方法平均数来填充；
print('处理前')
full.info()
#年龄
full['Age'] = full['Age'].fillna(full['Age'].mean())
#船票价格
full['Fare'] = full['Fare'].fillna(full['Fare'].mean())
print('处理后')
full.info()

#检查数据处理是否正常
full.head()

In [73]:
#将性别的值映射为数值，男性（male）对应1，女性（female）对应0
sex_set=set(full['Sex'])
full['Sex']=full['Sex'].map({'female': 0, 'male': 1}).astype(int)

In [74]:
#获取原始训练数据集和预测数据集
#原始数据集有891行
sourceRow = 891
#原始数据集：特征
s_X = full.loc[0:sourceRow-1,:]
source_X = s_X.drop('Survived',axis='columns')
#原始数据集：标签
source_y = full.loc[0:sourceRow-1,'Survived']
#预测数据集：特征
p_X = full.loc[sourceRow:,:]
pred_X = p_X.drop('Survived',axis='columns')

In [75]:
source_X

,Pclass,Sex,Age,Fare
0,3,1,22.000000,7.2500
1,1,0,38.000000,71.2833
2,3,0,26.000000,7.9250
3,1,0,35.000000,53.1000
4,3,1,35.000000,8.0500
...,...,...,...,...
886,2,1,27.000000,13.0000
887,1,0,19.000000,30.0000
888,3,0,29.881138,23.4500
889,1,1,26.000000,30.0000


In [76]:
source_y

0      0.0
1      1.0
2      1.0
3      1.0
4      0.0
      ... 
886    0.0
887    1.0
888    0.0
889    1.0
890    0.0
Name: Survived, Length: 891, dtype: float64

In [77]:
pred_X

,Pclass,Sex,Age,Fare
891,3,1,34.500000,7.8292
892,3,0,47.000000,7.0000
893,2,1,62.000000,9.6875
894,3,1,27.000000,8.6625
895,3,0,22.000000,12.2875
...,...,...,...,...
1304,3,1,29.881138,8.0500
1305,1,0,39.000000,108.9000
1306,3,1,38.500000,7.2500
1307,3,1,29.881138,8.0500


In [78]:
#建立模型用的训练数据集和预测数据集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(source_X, source_y, test_size = 0.25,random_state =0)

In [79]:
#建立分类朴素贝叶斯模型
from sklearn.naive_bayes import CategoricalNB
model = CategoricalNB()
model.fit(X_train, y_train)

CategoricalNB(alpha=1.0, class_prior=None, fit_prior=True)

In [81]:
#预测
pclassNo = int(input("Enter Person's Pclass number: "))
gender = int(input("Enter Person's Gender 0-female 1-male(0 or 1): "))
age = int(input("Enter Person's Age: "))
fare = float(input("Enter Person's Fare: "))
person = [[pclassNo,gender,age,fare]]
result = model.predict(person)


print('By Categorical Naive Bayes Method:')
if result == 1:
  print("Person might  Survive")
else:
  print("Person might not Survive")

Enter Person's Pclass number: 0
Enter Person's Gender 0-female 1-male(0 or 1): 1
Enter Person's Age: 25
Enter Person's Fare: 2
By Categorical Naive Bayes Method:
Person might not Survive


In [82]:
#去除测试集的标签乘客编号
y_test=test_y.drop('PassengerId',axis='columns')

In [83]:
#对预测数据进行预测
y_pred = model.predict(pred_X)
print(np.column_stack((y_pred,y_test)))

[[0. 0.]
 [0. 1.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [1. 0.]
 [0. 1.]
 [1. 1.]
 [1. 0.]
 [0. 0.]
 [1. 1.]
 [0. 1.]
 [0. 1.]
 [0. 0.]
 [1. 1.]
 [0. 1.]
 [1. 1.]
 [0. 0.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 1.]
 [1. 1.]
 [1. 1.]
 [0. 0.]
 [0. 1.]
 [0. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [1. 0.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 1.]
 [1. 1.]
 [0. 1.]
 [0. 1.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [0. 1.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [1. 1.]
 [1. 0.]
 [0. 0.]
 [1. 1.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [0. 1.]
 [0. 1.]
 [0. 0.]
 [1. 1.]
 [0. 0.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [1. 1.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 

In [84]:
#计算预测正确率
from sklearn.metrics import accuracy_score
print("Accuracy of the Model (Categorical Naive Bayes Method): {0}%".format(accuracy_score(y_test, y_pred)*100))

Accuracy of the Model (Categorical Naive Bayes Method): 81.10047846889952%
